### Datenaufbereitung für die Gehaltsdaten

#### 1. Import der benötigten Bibliotheken

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import re

#### 2. Import der Daten

In [2]:
file_path = 'ds_salaries.csv'
data = pd.read_csv(file_path)
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


#### 3. Erste Blicke auf die Daten

In [3]:

data_overview = {
    "first_five_rows": data.head(),
    "data_summary": data.describe(include='all'),
    "missing_values": data.isnull().sum(),
    "data_types": data.dtypes
}

data_overview


{'first_five_rows':    work_year experience_level employment_type                 job_title  \
 0       2023               SE              FT  Principal Data Scientist   
 1       2023               MI              CT               ML Engineer   
 2       2023               MI              CT               ML Engineer   
 3       2023               SE              FT            Data Scientist   
 4       2023               SE              FT            Data Scientist   
 
    salary salary_currency  salary_in_usd employee_residence  remote_ratio  \
 0   80000             EUR          85847                 ES           100   
 1   30000             USD          30000                 US           100   
 2   25500             USD          25500                 US           100   
 3  175000             USD         175000                 CA           100   
 4  120000             USD         120000                 CA           100   
 
   company_location company_size  
 0               E

#### 4. Kategorische Werte in numerische Werte umwandeln

Kategorische Werte: Folgende Spalten enthalten kategorische Daten: 
**experience_level, employment_type, job_title, salary_currency, employee_residence, company_location, company_size.**
Diese müssen in numerische Werte umgewandelt werden, da lineare Regressionsmodelle mit numerischen Daten arbeiten. Die Umwandlung kann durch Techniken wie One-Hot-Encoding oder Label-Encoding erfolgen.

In [4]:
# Auswahl der kategorischen Spalten
categorical_columns = data.select_dtypes(include=['object']).columns
print(categorical_columns)

# One-Hot-Encoding für kategorische Daten
encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(data[categorical_columns])


encoded_columns = encoder.get_feature_names_out(categorical_columns)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)


prepared_data = data.drop(columns=categorical_columns).join(encoded_df)


prepared_data_overview = {
    "first_five_rows": prepared_data.head(),
    "data_summary": prepared_data.describe(),
    "data_types": prepared_data.dtypes
}

prepared_data_overview

Index(['experience_level', 'employment_type', 'job_title', 'salary_currency',
       'employee_residence', 'company_location', 'company_size'],
      dtype='object')


c:\Users\csang\miniconda3\envs\fhgr\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'first_five_rows':    work_year  salary  salary_in_usd  remote_ratio  experience_level_EN  \
 0       2023   80000          85847           100                  0.0   
 1       2023   30000          30000           100                  0.0   
 2       2023   25500          25500           100                  0.0   
 3       2023  175000         175000           100                  0.0   
 4       2023  120000         120000           100                  0.0   
 
    experience_level_EX  experience_level_MI  experience_level_SE  \
 0                  0.0                  0.0                  1.0   
 1                  0.0                  1.0                  0.0   
 2                  0.0                  1.0                  0.0   
 3                  0.0                  0.0                  1.0   
 4                  0.0                  0.0                  1.0   
 
    employment_type_CT  employment_type_FL  ...  company_location_SI  \
 0                 0.0                 0.

#### 5. Standardisierung der Daten

Die numerischen Spalten **work_year, salary und remote_ratio** wurden standardisiert, um einen Mittelwert von 0 und eine Standardabweichung von 1 zu erreichen. Dies hilft dabei, die Daten auf eine gemeinsame Skala zu bringen und die Leistung des linearen Regressionsmodells zu verbessern.

In [5]:


# Auswahl der zu skalierenden numerischen Spalten (außer der Zielvariablen)
numerical_columns = ['work_year', 'salary', 'remote_ratio']


scaler = StandardScaler()
scaled_data = scaler.fit_transform(prepared_data[numerical_columns])


scaled_df = pd.DataFrame(scaled_data, columns=numerical_columns)

final_data = prepared_data.drop(columns=numerical_columns).join(scaled_df)


final_data_overview = {
    "first_five_rows": final_data.head(),
    "data_summary": final_data.describe(),
    "data_types": final_data.dtypes
}

final_data_overview

{'first_five_rows':    salary_in_usd  experience_level_EN  experience_level_EX  \
 0          85847                  0.0                  0.0   
 1          30000                  0.0                  0.0   
 2          25500                  0.0                  0.0   
 3         175000                  0.0                  0.0   
 4         120000                  0.0                  0.0   
 
    experience_level_MI  experience_level_SE  employment_type_CT  \
 0                  0.0                  1.0                 0.0   
 1                  1.0                  0.0                 1.0   
 2                  1.0                  0.0                 1.0   
 3                  0.0                  1.0                 0.0   
 4                  0.0                  1.0                 0.0   
 
    employment_type_FL  employment_type_FT  employment_type_PT  \
 0                 0.0                 1.0                 0.0   
 1                 0.0                 0.0                 

#### 6. Entfernen von Spalte salary

In [6]:
# Entfernen der Spalte 'salary'
final_data_updated = final_data.drop(columns=['salary'])

# Überblick über den aktualisierten Datensatz
final_data_updated_overview = {
    "first_five_rows": final_data_updated.head(),
    "data_summary": final_data_updated.describe(),
    "data_types": final_data_updated.dtypes
}

final_data_updated_overview


{'first_five_rows':    salary_in_usd  experience_level_EN  experience_level_EX  \
 0          85847                  0.0                  0.0   
 1          30000                  0.0                  0.0   
 2          25500                  0.0                  0.0   
 3         175000                  0.0                  0.0   
 4         120000                  0.0                  0.0   
 
    experience_level_MI  experience_level_SE  employment_type_CT  \
 0                  0.0                  1.0                 0.0   
 1                  1.0                  0.0                 1.0   
 2                  1.0                  0.0                 1.0   
 3                  0.0                  1.0                 0.0   
 4                  0.0                  1.0                 0.0   
 
    employment_type_FL  employment_type_FT  employment_type_PT  \
 0                 0.0                 1.0                 0.0   
 1                 0.0                 0.0                 

#### 7. Aufteilung der Daten in Trainings- und Testdaten

In [7]:
# Definieren der Merkmale und der Zielvariable
X = final_data_updated.drop('salary_in_usd', axis=1)
y = final_data_updated['salary_in_usd']

# Aufteilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Größe der aufgeteilten Datensätze
train_test_split_sizes = {
    "X_train_shape": X_train.shape,
    "X_test_shape": X_test.shape,
    "y_train_shape": y_train.shape,
    "y_test_shape": y_test.shape
}

train_test_split_sizes


{'X_train_shape': (3004, 276),
 'X_test_shape': (751, 276),
 'y_train_shape': (3004,),
 'y_test_shape': (751,)}

#### 8. Speichern der Daten

In [8]:
savepathX_train = 'AufbereiteteDaten/X_trainSalary.csv'
savepathX_test = 'AufbereiteteDaten/X_testSalary.csv'
savepathy_train = 'AufbereiteteDaten/y_trainSalary.csv'
savepathy_test = 'AufbereiteteDaten/y_testSalary.csv'

savefullData = 'AufbereiteteDaten/fullDataSalary.csv'

X_train.to_csv(savepathX_train, index=False)
X_test.to_csv(savepathX_test, index=False)
y_train.to_csv(savepathy_train, index=False)
y_test.to_csv(savepathy_test, index=False)

final_data_updated.to_csv(savefullData, index=False)